In [2]:
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from sklearn import datasets
import numpy as np
from scipy.optimize import basinhopping, differential_evolution

In [3]:
iris = datasets.load_iris()
X = iris.data
y = iris.target



n_sample = len(X)

np.random.seed(0)
order = np.random.permutation(n_sample)
X = X[order]
y = y[order].astype(np.float)

val_size=0.1
test_size=0.1

l_val=int(val_size*n_sample)
l_test=int(test_size*n_sample)
l_train=n_sample-l_val-l_test


X_train=X[:l_train]
y_train=y[:l_train]
X_val=X[l_train:l_val+l_train]
y_val=y[l_train:l_val+l_train]
X_test=X[l_val+l_train:]
y_test=y[l_val+l_train:]



In [4]:
def f(x):
    clf = SVC(C=x[0], gamma=x[1], kernel='rbf')
    clf.fit(X_train, y_train)
    return (-clf.score(X_val,y_val))

# BASIN-HOPPING

In [5]:
class RandomDisplacementBounds(object):
    """random displacement with bounds:  see: https://stackoverflow.com/a/21967888/2320035
        Modified! (dropped acceptance-rejection sampling for a more specialized approach)
    """
    def __init__(self, xmin, xmax, stepsize=0.5):
        self.xmin = xmin
        self.xmax = xmax
        self.stepsize = stepsize

    def __call__(self, x):
        """take a random step but ensure the new position is within the bounds """
        min_step = np.maximum(self.xmin - x, -self.stepsize)
        max_step = np.minimum(self.xmax - x, self.stepsize)

        random_step = np.random.uniform(low=min_step, high=max_step, size=x.shape)
        xnew = x + random_step

        return xnew

In [6]:
bc = (0.1*10**(-30), float("inf"))
bg = (0.1*10**(-30), float("inf"))
bounds = [bc,bg]

x0vect=[[1,0.1], [2,3], [1000, 10]]

In [7]:
minimizer_kwargs = {"method":"L-BFGS-B", "bounds": bounds}
bounded_step = RandomDisplacementBounds(np.array([b[0] for b in bounds]), np.array([b[1] for b in bounds]))

ret={}
for i, x0 in enumerate(x0vect):
    ret[i] = basinhopping(f, x0, minimizer_kwargs=minimizer_kwargs, take_step=bounded_step, niter=1000)

max=ret[0]
for i, x0 in enumerate(x0vect): 
    if max.fun>ret[i].fun:
        max=ret[i]

print("global maximum: C = %.4f, gamma = %.4f, f(x0) = %.4f (evaluation set)" % (max.x[0], max.x[1],(-max.fun)))


global maximum: C = 1.0920, gamma = 0.3434, f(x0) = 0.9333 (evaluation set)


In [8]:
#Testing the value on the test set

c=max.x[0]
g=max.x[1]
clf = SVC(C=c, gamma=g, kernel='rbf')
clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

print(clf.predict(X_test), y_test)

0.9333333333333333
[1. 2. 2. 0. 1. 1. 2. 1. 0. 0. 0. 2. 1. 2. 0.] [1. 2. 2. 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 2. 0.]


# DIFFERETIAL EVOLUTION

In [16]:
bc = (0.1*10**(-30), 100)
bg = (0.1*10**(-30), 100)
bounds = [bc,bg]

res = differential_evolution(f, bounds)

print("global maximum: C = %.4f, gamma = %.4f, f(x0) = %.4f  (evaluation set)" % (res.x[0], res.x[1],(-res.fun)))   #use population for a better result. the output can change every time the algorithm is launched


global maximum: C = 1.3908, gamma = 2.2481, f(x0) = 0.9333  (evaluation set)


In [17]:
#Testing the value on the test set

c=res.x[0]
g=res.x[1]
clf = SVC(C=c, gamma=g, kernel='rbf')
clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

print(clf.predict(X_test), y_test)

0.9333333333333333
[1. 2. 2. 0. 1. 1. 2. 1. 0. 0. 0. 2. 1. 2. 0.] [1. 2. 2. 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 2. 0.]


# Dual Annealing

In [11]:
from scipy.optimize import dual_annealing

In [12]:
bc = (0.1*10**(-30), 100)
bg = (0.1*10**(-30), 100)
bounds = [bc,bg]

res = dual_annealing(f, bounds)

print("global maximum: C = %.4f, gamma = %.4f, f(x0) = %.4f" % (res.x[0], res.x[1],(-res.fun)))   #use population for a better result. the output can change every time the algorithm is launched


global maximum: C = 0.1988, gamma = 0.9719, f(x0) = 0.9333


In [13]:
#Testing the value on the test set

c=res.x[0]
g=res.x[1]
clf = SVC(C=c, gamma=g, kernel='rbf')
clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

print(clf.predict(X_test), y_test)

1.0
[1. 2. 2. 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 2. 0.] [1. 2. 2. 0. 1. 1. 1. 1. 0. 0. 0. 2. 1. 2. 0.]
